# **Machine Learning Forex TWAP Solution using scikit-learn**

This notebook demonstrates Machine Learning solution to predict TWAP direction for given currency pair based on historical prices

## Problem Formulation

In this example, we will use Allison Horst's Palmer Penguins dataset, available here: https://github.com/allisonhorst/palmerpenguins.

The dataset contains data on 344 penguins. There are three different species of penguins in the dataset, collected from three separate islands in the Palmer Archipelago, Antarctica. 

In [10]:
#tables and visualizations
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#machine learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler
from sklearn import config_context
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [8]:
%ls /eurusd.csv


/eurusd.csv


## Load Data

Here we first load the data into python using pandas and read it in as a pandas dataframe which is the format which we will use throughout the example. 

In [30]:
fx_volume = pd.read_csv("/eurusd.csv")
display(fx_volume.head())
fx_volume.info()

,<TICKER>,<DTYYYYMMDD>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,EURUSD,20180401,21:00,1.23199,1.23230,1.23165,1.23204,469600000
1,EURUSD,20180401,22:00,1.23204,1.23217,1.23132,1.23172,4095050000
2,EURUSD,20180401,23:00,1.23172,1.23206,1.23124,1.23125,3291760001
3,EURUSD,20180402,00:00,1.23127,1.23217,1.23107,1.23217,5418240002
4,EURUSD,20180402,01:00,1.23219,1.23282,1.23214,1.23263,4164520003


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507 entries, 0 to 506
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   <TICKER>      507 non-null    object 
 1   <DTYYYYMMDD>  507 non-null    int64  
 2   <TIME>        507 non-null    object 
 3   <OPEN>        507 non-null    float64
 4   <HIGH>        507 non-null    float64
 5   <LOW>         507 non-null    float64
 6   <CLOSE>       507 non-null    float64
 7   <VOL>         507 non-null    int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 31.8+ KB


In [52]:
fx_volume['<AVG_PRICE>'] = (fx_volume['<HIGH>'] + fx_volume['<LOW>'] + fx_volume['<CLOSE>'] + fx_volume['<OPEN>'])/4 
fx_volume['<PV>'] = fx_volume['<VOL>'] * fx_volume['<AVG_PRICE>'] 
display(fx_volume.head())
fx_volume.info()

,<TICKER>,<DTYYYYMMDD>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,AVG_PRICE,<AVG_PRICE>,<PV>,<KEY>
0,EURUSD,20180401,21:00,1.23199,1.23230,1.23165,1.23204,469600000,1.231995,1.231995,5.785449e+08,EURUSD 0 20180401\n1 20180401\n2 ...
1,EURUSD,20180401,22:00,1.23204,1.23217,1.23132,1.23172,4095050000,1.231813,1.231813,5.044334e+09,EURUSD 0 20180401\n1 20180401\n2 ...
2,EURUSD,20180401,23:00,1.23172,1.23206,1.23124,1.23125,3291760001,1.231567,1.231567,4.054025e+09,EURUSD 0 20180401\n1 20180401\n2 ...
3,EURUSD,20180402,00:00,1.23127,1.23217,1.23107,1.23217,5418240002,1.231670,1.231670,6.673484e+09,EURUSD 0 20180401\n1 20180401\n2 ...
4,EURUSD,20180402,01:00,1.23219,1.23282,1.23214,1.23263,4164520003,1.232445,1.232445,5.132542e+09,EURUSD 0 20180401\n1 20180401\n2 ...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507 entries, 0 to 506
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   <TICKER>      507 non-null    object 
 1   <DTYYYYMMDD>  507 non-null    int64  
 2   <TIME>        507 non-null    object 
 3   <OPEN>        507 non-null    float64
 4   <HIGH>        507 non-null    float64
 5   <LOW>         507 non-null    float64
 6   <CLOSE>       507 non-null    float64
 7   <VOL>         507 non-null    int64  
 8   AVG_PRICE     507 non-null    float64
 9   <AVG_PRICE>   507 non-null    float64
 10  <PV>          507 non-null    float64
 11  <KEY>         507 non-null    object 
dtypes: float64(7), int64(2), object(3)
memory usage: 47.7+ KB


In [55]:
grouped = fx_volume['<PV>'].groupby(fx_volume['<TICKER>']).agg(['sum','count'])
display(grouped.head())

,sum,count
<TICKER>,,
EURUSD,5.967096e+12,507


# Compute PRICE_VOLUME
##AVG_PRICE = (High +low + Close)/3
##PRICE_VOLUME = AVG_PRICE*VOLUME
  fx_volume["AVG_PRICE"] = (fx_volume['<HIGH>']+fx_volume['<LOW>']+fx_volume['<CLOSE>'])/3
  fx_volume["PV"]=fx_volume['<AVG_PRICE>']*fx_volume['<VOL>']
  display(fx_volume.head())
  fx_volume.info()  



# Compute PRICE_VOLUME
##AVG_PRICE = (High +low + Close)/3
##PRICE_VOLUME = AVG_PRICE*VOLUME
  fx_volume["AVG_PRICE"] = (fx_volume['<HIGH>']+fx_volume['<LOW>']+fx_volume['<CLOSE>'])/3
  fx_volume["PV"]=fx_volume['<AVG_PRICE>']*fx_volume['<VOL>']
  display(fx_volume.head())
  fx_volume.info()  



In [14]:
fx_volume.isna().sum()

<TICKER>        0
<DTYYYYMMDD>    0
<TIME>          0
<OPEN>          0
<HIGH>          0
<LOW>           0
<CLOSE>         0
<VOL>           0
dtype: int64

## Data cleaning and EDA

We can now explore our data. We leave this exercise to the reader. For now, we can observe that there are a few NA values which will likely need imputation. We'll wait for this step so that we can put it within our training loop. For now, we'll just drop all of the sex NAs out of the dataframe.

In [15]:
fx_volume = fx_volume.dropna(subset=['volume'])
fx_volume.shape

KeyError: ignored

In [ ]:
class_column = 'london_date'
random_seed = 2435

X_train, X_test, y_train, y_test = train_test_split(fx_volume.drop(columns=class_column), penguins[class_column],
                                                   test_size=0.25, random_state=random_seed, stratify=penguins[class_column])

Quick sanity check to make sure that everything seems correct:

In [ ]:
# X Train
print('On X train: ')
print('X train dimensions: ', X_train.shape)
display(X_train.head())

# X test
print('\nOn X test: ')
print('X test dimensions: ', X_test.shape)
display(X_test.head())

In [ ]:
# X Train
print('On y train: ')
print('y train dimensions: ', y_train.shape)
display(y_train.head())

# X test
print('\nOn y test: ')
print('y test dimensions: ', y_test.shape)
display(y_test.head())

## Establish the training pipeline

We can now establish the training pipeline for our models. Since this is a process we would need to repeat several times, it's good to essentially functionalize the process so we do not need to re-write redundant code. Here, we can impute some values that were missing, and encode any categorical values. Note that these pipelines will change according to the model and methodology you choose - additionally, the pipelines will also change depending on the data types of the columns in your dataset. 

In [ ]:
#individual pipelines for differing datatypes
cat_pipeline = Pipeline(steps=[('cat_impute', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                               ('onehot_cat', OneHotEncoder(drop='if_binary'))])
num_pipeline = Pipeline(steps=[('impute_num', SimpleImputer(missing_values=np.nan, strategy='mean')),
                               ('scale_num', StandardScaler())])

In [ ]:
#establish preprocessing pipeline by columns
preproc = ColumnTransformer([('cat_pipe', cat_pipeline, make_column_selector(dtype_include=object)),
                             ('num_pipe', num_pipeline, make_column_selector(dtype_include=np.number))],
                             remainder='passthrough')

In [ ]:
#generate the whole modeling pipeline with preprocessing
pipe = Pipeline(steps=[('preproc', preproc),
                       ('mdl', LogisticRegression(penalty='elasticnet', solver='saga', tol=0.01))])

#visualization for steps
with config_context(display='diagram'):
    display(pipe)

## Cross-validation with hyperparameter tuning

Now that we have our pipelines, we can now use this as part of cross validation and hyperparameter tuning.

In [ ]:
tuning_grid = {'mdl__l1_ratio' : np.linspace(0,1,5),
               'mdl__C': np.logspace(-1, 6, 3) }
grid_search = GridSearchCV(pipe, param_grid = tuning_grid, cv = 5, return_train_score=True)

In [ ]:
tuning_grid

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_score_)
grid_search.best_params_

In [ ]:
pd.DataFrame(grid_search.cv_results_)

## Final fit

The final fit here is already present in the generated model due to the way we set our parameters in the grid search. If we want to look at the performance, we can do so. Here is a non-helpful description of the best model:

In [ ]:
grid_search.best_estimator_

## Variable importance

Now we assess the importance in the selected model to reveal any potential insights.

In [ ]:
grid_search.classes_

In [ ]:
vip = grid_search.best_estimator_['mdl'].coef_[0]
vip

In [ ]:
#get names in correct preproc order
cat_names = grid_search.best_estimator_.named_steps['preproc'].transformers_[0][1].named_steps['onehot_cat'].get_feature_names()
num_names = grid_search.best_estimator_.named_steps['preproc'].transformers_[1][2]

#create df with vip info
coef_info = pd.DataFrame({'feat_names':np.hstack([cat_names, num_names]), 'vip': vip})

#get sign and magnitude information
coef_info = coef_info.assign(coef_mag = abs(coef_info['vip']),
                             coef_sign = np.sign(coef_info['vip']))

#sort and plot
coef_info = coef_info.set_index('feat_names').sort_values(by='coef_mag', ascending=False)
sns.barplot(y=coef_info.index, x='coef_mag', hue='coef_sign', data=coef_info, orient='h', dodge=False);

## Performance metrics on test data


Here, we can see the performance of the model, which is pretty nice! We can also look into different scores specifically for more insight into the performance.

In [ ]:
print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

In [ ]:
cm = confusion_matrix(y_test, grid_search.best_estimator_.predict(X_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=grid_search.classes_)
disp.plot()

plt.show()

## Try it yourself!

Now that we've seen the power of pipelines in sklearn, let's now try implementing our own pipelines.

In [ ]:
# Try implementing a pipeline where we use median imputation for numeric columns instead of mean imputation.

#individual pipelines for differing datatypes
cat_pipeline = Pipeline(steps=[('cat_impute', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                               ('onehot_cat', OneHotEncoder(drop='if_binary'))])
num_pipeline = Pipeline(steps=[('impute_num', SimpleImputer(missing_values=np.nan, strategy='median')),
                               ('scale_num', StandardScaler())])

#establish preprocessing pipeline by columns
preproc = ColumnTransformer([('cat_pipe', cat_pipeline, make_column_selector(dtype_include=object)),
                             ('num_pipe', num_pipeline, make_column_selector(dtype_include=np.number))],
                             remainder='passthrough')

With this new pipeline, now train a Random Forest model. Refer to the documentation for the parameters for the random forest classifier here: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Is the performance better? 

In [ ]:
#generate the whole modeling pipeline with preprocessing
pipe = Pipeline(steps=[('preproc', preproc),
                       ('mdl', RandomForestClassifier())])

#visualization for steps
with config_context(display='diagram'):
    display(pipe)

Now perform cross validation and modify the n_estimators parameter to values of [100, 200,500] and max_depth parameter to values of [10,15,50] for the random forest classifier for hyperparameter tuning.

In [ ]:
tuning_grid = {'mdl__n_estimators' : [100, 200 ,500],
               'mdl__max_depth': [10, 15, 20] }
grid_search = GridSearchCV(pipe, param_grid = tuning_grid, cv = 5, return_train_score=True)

In [ ]:
tuning_grid

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_score_)
grid_search.best_params_

In [ ]:
pd.DataFrame(grid_search.cv_results_)

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.classes_

In [ ]:
vip = grid_search.best_estimator_['mdl'].feature_importances_
vip

In [ ]:
#get names in correct preproc order
cat_names = grid_search.best_estimator_.named_steps['preproc'].transformers_[0][1].named_steps['onehot_cat'].get_feature_names()
num_names = grid_search.best_estimator_.named_steps['preproc'].transformers_[1][2]

#create df with vip info
coef_info = pd.DataFrame({'feat_names':np.hstack([cat_names, num_names]), 'vip': vip})

#get sign and magnitude information
coef_info = coef_info.assign(coef_mag = abs(coef_info['vip']),
                             coef_sign = np.sign(coef_info['vip']))

#sort and plot
coef_info = coef_info.set_index('feat_names').sort_values(by='coef_mag', ascending=False)
sns.barplot(y=coef_info.index, x='coef_mag', hue='coef_sign', data=coef_info, orient='h', dodge=False);

In [ ]:
print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

In [ ]:
cm = confusion_matrix(y_test, grid_search.best_estimator_.predict(X_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=grid_search.classes_)
disp.plot()

plt.show()